In [1]:
import pandas as pd

In [2]:
import torch
from torch import nn

将torch.Tensor转换为numpy.ndarray

In [3]:
matrix = torch.randn(4,3)
mat_np = matrix.numpy()
# 直接使用DataFrame便可将矩阵转为pandas的DataFrame
mat_pd = pd.DataFrame(mat_np)
print(mat_pd)

          0         1         2
0 -1.555229 -0.072155  1.158482
1 -1.829319 -0.028049  0.192595
2 -3.047824  0.889358 -1.583263
3 -0.536973 -0.312071 -1.320867


DataFrame的字典用法，使用属性列生成DataFrame

In [17]:
pd.DataFrame({1: [1, 2, 3], 2: [4, 5, 6]})

,1,2
0,1,4
1,2,5
2,3,6


从pandas中，切分出features和labels

方式一

In [23]:
features_1,labels_1 = mat_pd.iloc[:,:-1],mat_pd.iloc[:,-1]
features_1.shape,labels_1.shape

((4, 2), (4,))

方式二

In [5]:
features_2,labels_2 = mat_pd[mat_pd.columns[:-1]],mat_pd[mat_pd.columns[-1]]
features_2.shape,labels_2.shape

((4, 2), (4,))

更换pandas的columns

In [9]:
import copy
new_pd = copy.copy(mat_pd)
id(new_pd) == id(mat_pd)

False

In [12]:
id(new_pd[0]) == id(mat_pd[0])

False

In [16]:
new_pd.columns = ['a','b','c']
new_pd

,a,b,c
0,-1.000000,-0.072155,1.158482
1,-1.829319,-0.028049,0.192595
2,-3.047824,0.889358,-1.583263
3,-0.536973,-0.312071,-1.320867


## selectKBest

In [20]:
true_w = torch.Tensor([15,12,10,4,2,1,0.1,-2,-5,-10])
def generate_data(num_len):
    """根据true_w生成数据"""
    x = torch.normal(0,1,(num_len,len(true_w)))
    data = true_w * x + torch.normal(0,0.01,(len(true_w),))
    return x,torch.sum(data, dim=-1)
features,labels = generate_data(1024)
data = torch.concat([features,labels.reshape(-1,1)],dim=-1)
# 将Tensor转为pandas的DataFrame
csv_data = pd.DataFrame(data)
# 将数据写入到本地的csv文件中
csv_data.to_csv('data.csv',index=False)

读取csv文件，选出k个权重(绝对值)大的特征，

In [21]:
def get_KBestFeatures(features,labels,k=None):
    """
        获取最好的k个特征,k默认返回所有特征,
        features和labels需要是pandas的DataFrame类型
        @features: 特征矩阵
        @labels: regression的标签
    """
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import f_regression
    features = (features - features.min()) / (features.max() - features.min())
    if k is None:
        k = features.shape[-1]
    bestfeatures = SelectKBest(score_func=f_regression,k=k)
    fit = bestfeatures.fit(features,labels)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(features.columns)
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  # naming the dataframe columns
    return featureScores.nlargest(k,'Score')   # return k best features

读取csv

In [24]:
data_pd = pd.read_csv('data.csv')
features,labels = data_pd.iloc[:,:-1],data_pd.iloc[:,-1]
features.shape,labels.shape

((1024, 10), (1024,))

In [25]:
get_KBestFeatures(features,labels)

,Specs,Score
0,0,576.288470
1,1,277.147010
9,9,203.042308
2,2,193.348680
3,3,34.702173
8,8,34.494189
4,4,18.866679
7,7,18.269984
5,5,3.108809
6,6,0.071812


排序结果基本符合true_w的绝对值排序结果